In [1]:
import random
import time
import statistics

def contar_colisoes(estado):
    """
    estado: lista de 8 inteiros (0..7), índice = coluna, valor = linha.
    Retorna o número de pares de rainhas em conflito.
    """
    n = len(estado)
    colisoes = 0
    for i in range(n):
        for j in range(i + 1, n):
            # mesma linha
            if estado[i] == estado[j]:
                colisoes += 1
            # mesma diagonal
            elif abs(estado[i] - estado[j]) == abs(i - j):
                colisoes += 1
    return colisoes


In [3]:
N_COLUNAS = 8
TAM_POP = 20
# Representação e População Inicial

def gerar_individuo():
    # vetor de 8 posições, cada posição é a linha da rainha na coluna
    return [random.randint(0, N_COLUNAS - 1) for _ in range(N_COLUNAS)]

def gerar_populacao_inicial():
    return [gerar_individuo() for _ in range(TAM_POP)]


In [4]:
def fitness(estado):
    # Roleta
    # quanto menor o número de colisões, melhor; vamos transformar em valor a maximizar
    col = contar_colisoes(estado)
    return 1 / (1 + col)  # fitness em (0,1], col=0 dá 1.0

def selecionar_por_roleta(populacao):
    # calcula fitness de todos
    valores = [fitness(ind) for ind in populacao]
    soma = sum(valores)
    if soma == 0:
        # todos iguais, escolhe uniforme
        return random.choice(populacao)
    # sorteia ponto na roleta
    r = random.uniform(0, soma)
    acumulado = 0
    for ind, val in zip(populacao, valores):
        acumulado += val
        if acumulado >= r:
            return ind
    return populacao[-1]


In [5]:
TAXA_CRUZAMENTO = 0.8
  #Cruzamento de 1 ponto (taxa 80%)

def cruzar_1_ponto(pai1, pai2):
    ponto = random.randint(1, N_COLUNAS - 1)
    filho1 = pai1[:ponto] + pai2[ponto:]
    filho2 = pai2[:ponto] + pai1[ponto:]
    return filho1, filho2

def gerar_filhos(populacao):
    filhos = []
    # vamos gerar filhos até ter TAM_POP indivíduos na nova população
    while len(filhos) < TAM_POP:
        pai1 = selecionar_por_roleta(populacao)
        pai2 = selecionar_por_roleta(populacao)
        if random.random() < TAXA_CRUZAMENTO:
            f1, f2 = cruzar_1_ponto(pai1, pai2)
        else:
            f1, f2 = pai1[:], pai2[:]
        filhos.append(f1)
        if len(filhos) < TAM_POP:
            filhos.append(f2)
    return filhos[:TAM_POP]


In [6]:
TAXA_MUTACAO = 0.03

def mutar_bit_flip(individuo):
    # cada posição tem chance de 3% de ser mudada para outra linha
    for i in range(len(individuo)):
        if random.random() < TAXA_MUTACAO:
            linhas = list(range(N_COLUNAS))
            linhas.remove(individuo[i])
            individuo[i] = random.choice(linhas)
    return individuo

def mutar_populacao(populacao):
    return [mutar_bit_flip(ind[:]) for ind in populacao]


In [8]:
MAX_GERACOES = 1000
# Elitismo e laço principal do algoritmo genético
def algoritmo_genetico():
    populacao = gerar_populacao_inicial()
    geracao = 0

    inicio_tempo = time.time()

    # avalia melhor indivíduo inicial
    melhor = min(populacao, key=contar_colisoes)
    melhor_fit = contar_colisoes(melhor)

    while geracao < MAX_GERACOES and melhor_fit > 0:
        geracao += 1

        # gera filhos por cruzamento
        filhos = gerar_filhos(populacao)
        # aplica mutação
        filhos = mutar_populacao(filhos)

        # avalia filhos
        filhos_ordenados = sorted(filhos, key=contar_colisoes)

        # elitismo: garante que o melhor da geração anterior sobreviva
        melhor_atual = min(populacao, key=contar_colisoes)
        if contar_colisoes(melhor_atual) < melhor_fit:
            melhor = melhor_atual
            melhor_fit = contar_colisoes(melhor)

        # monta nova população: melhor antigo + melhores filhos
        nova_pop = [melhor] + filhos_ordenados[:-1]
        populacao = nova_pop

        # atualiza melhor global com base na nova população
        cand = min(populacao, key=contar_colisoes)
        cand_fit = contar_colisoes(cand)
        if cand_fit < melhor_fit:
            melhor, melhor_fit = cand, cand_fit

    tempo_execucao = time.time() - inicio_tempo
    # geracao é o número de gerações efetivamente usadas
    return melhor, melhor_fit, geracao, tempo_execucao


In [9]:
resultados_iter = []
resultados_tempo = []

# Executar 50 vezes e calcular estatísticas
NUM_EXECUCOES = 50

for _ in range(NUM_EXECUCOES):
    melhor, melhor_fit, geracoes, tempo = algoritmo_genetico()
    resultados_iter.append(geracoes)
    resultados_tempo.append(tempo)

# listar execuções (opcional)
for i, (g, t) in enumerate(zip(resultados_iter, resultados_tempo), start=1):
    print(f"Execução {i:2d}: gerações = {g:4d} | tempo = {t:.6f} s")


Execução  1: gerações = 1000 | tempo = 2.238795 s
Execução  2: gerações =  307 | tempo = 0.668846 s
Execução  3: gerações = 1000 | tempo = 2.233483 s
Execução  4: gerações =  165 | tempo = 0.354888 s
Execução  5: gerações =    7 | tempo = 0.014960 s
Execução  6: gerações =   28 | tempo = 0.061902 s
Execução  7: gerações = 1000 | tempo = 2.606705 s
Execução  8: gerações =   28 | tempo = 0.113046 s
Execução  9: gerações = 1000 | tempo = 2.929628 s
Execução 10: gerações =  188 | tempo = 0.423478 s
Execução 11: gerações = 1000 | tempo = 2.189892 s
Execução 12: gerações =  108 | tempo = 0.231825 s
Execução 13: gerações = 1000 | tempo = 2.184960 s
Execução 14: gerações =  121 | tempo = 0.260268 s
Execução 15: gerações = 1000 | tempo = 2.275584 s
Execução 16: gerações = 1000 | tempo = 3.047067 s
Execução 17: gerações = 1000 | tempo = 2.502343 s
Execução 18: gerações = 1000 | tempo = 2.269690 s
Execução 19: gerações = 1000 | tempo = 2.242375 s
Execução 20: gerações = 1000 | tempo = 2.203182 s


In [11]:
# Média e desvio padrão (letra b)
media_iter = statistics.mean(resultados_iter)
desvio_iter = statistics.pstdev(resultados_iter)

media_tempo = statistics.mean(resultados_tempo)
desvio_tempo = statistics.pstdev(resultados_tempo)

print("\nResumo estatístico (50 execuções):")
print(f"Média de gerações até parar: {media_iter:.2f}")
print(f"Desvio padrão das gerações: {desvio_iter:.2f}")
print(f"Média do tempo de execução (s): {media_tempo:.6f}")
print(f"Desvio padrão do tempo (s): {desvio_tempo:.6f}")



Resumo estatístico (50 execuções):
Média de gerações até parar: 699.10
Desvio padrão das gerações: 413.54
Média do tempo de execução (s): 1.718288
Desvio padrão do tempo (s): 1.075054


In [12]:
def melhores_solucoes_genetico(qtd=5):
    solucoes = []

    while len(solucoes) < qtd:
        melhor, melhor_fit, geracoes, tempo = algoritmo_genetico()

        if all(melhor != s[0] for s in solucoes):
            solucoes.append((melhor, melhor_fit))

    solucoes_ordenadas = sorted(solucoes, key=lambda x: x[1])
    return solucoes_ordenadas

cinco_melhores = melhores_solucoes_genetico()

print("Cinco melhores soluções distintas encontradas pelo algoritmo genético:")
for i, (ind, fit) in enumerate(cinco_melhores, start=1):
    print(f"Solução {i}: estado = {ind} | colisões = {fit}")


Cinco melhores soluções distintas encontradas pelo algoritmo genético:
Solução 1: estado = [4, 0, 7, 3, 1, 6, 2, 5] | colisões = 0
Solução 2: estado = [2, 4, 1, 7, 0, 6, 3, 5] | colisões = 0
Solução 3: estado = [1, 3, 6, 0, 7, 4, 0, 5] | colisões = 1
Solução 4: estado = [0, 3, 6, 2, 7, 1, 3, 5] | colisões = 1
Solução 5: estado = [1, 3, 5, 0, 2, 4, 2, 7] | colisões = 1
